*Created: 23 February 2022*<br>
*Last Update: 24 February 2022*<br>
*Author: Yasir Abdur Rohman*<br>
**Property of PT Indonesia Power & Lab Getaran & Diagnosis Mesin Undip**

---

# Pipeline Deployment on Notebook
Case: Local Bearing Problem

In [1]:
from estimate.vbm import VBM
from residual.residual import Residual
from utils.pihelper import PIHelper
from utils.transform import TransformData
from utils.model import Model
from utils.db import insert_to_table_runtime
from utils.threshold import get_actual_low, get_actual_high, get_sensor_id
from utils.threshold import get_residual_positive_threshold, get_residual_negative_threshold

from db.base import session
from db.diagnostic.models.model_fault import Fault
from db.assets.models.model_assets import Assets
from db.assets.models.model_asset_failure import AssetFailure
from db.diagnostic.models.model_model_tag import ModelTag
from db.diagnostic.models.model_priority import Priority
from db.diagnostic.models.model_diagnose_date import DiagnoseDate

from osisoft.pidevclub.piwebapi.models import PIAnalysis, PIItemsStreamValues, PIStreamValues, PITimedValue, PIRequest

import pandas as pd
import time
from datetime import timedelta, date, datetime

In [2]:
# connect client
pihelper = PIHelper()
client = pihelper.connect_client()

# setup paths
parent = "af:\\\\pi1\\SLA5."
sensor = ['Generator Gross Capacity',
          'Turbine Lube Oil Cooler Outlet Temperature',
          'Turbine.Bearing 1 Drain Oil Temperature',
          'Turbine.Bearing 1 Metal Temperature',
          'Turbine.Bearing 1X Vibration',
          'Turbine.Bearing 1Y Vibration']

paths = [parent+sensor[i] for i in range(len(sensor))]

In [3]:
# extract and transform data

def etl(paths, start_time, end_time, interval):
    # extract
    data = client.data.get_multiple_interpolated_values(paths, start_time=start_time, end_time=end_time, interval=interval)
    # transform
    transformer = TransformData()
    data = transformer.reduce_columns(data, sensor)
    data = transformer.transform(data)
    return data

In [9]:
start_time = '2020-04-04 17:30:00'
# end_time = '2020-01-01 00:01:00'
interval = '1m'

result = client.data.get_multiple_interpolated_values(paths, start_time=start_time, end_time=start_time, interval=interval)
transformer = TransformData()
data = transformer.reduce_columns(result, sensor)
data
# etl(paths, start_time, start_time, interval)

date                                                                        2020-04-04 17:30:00
Generator Gross Capacity                      {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Turbine Lube Oil Cooler Outlet Temperature    {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Turbine.Bearing 1 Drain Oil Temperature       {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Turbine.Bearing 1 Metal Temperature           {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Turbine.Bearing 1X Vibration                  {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Turbine.Bearing 1Y Vibration                  {'Name': 'Unit Down', 'Value': 213, 'IsSystem'...
Name: 0, dtype: object

In [27]:
type(dict()) == float

False

# Loop

In [5]:
# define asset and fault
fault = 'Local Bearing'
asset = session.query(Assets).get(2) #get asset->id=2

# Threshold
actual_low = get_actual_low(sensor)
actual_high = get_actual_high(sensor)
sensor_id = get_sensor_id(sensor)
residual_positive_threshold = get_residual_positive_threshold(sensor_id)
residual_negative_threshold = get_residual_negative_threshold(sensor_id)

In [6]:
point1 = client.point.get_by_path("\\\\PI1\SLA5.Turbine.recommendation prescriptive prediction", None)
point2 = client.point.get_by_path("\\\\PI1\SLA5.Turbine.priority prescriptive prediction", None)
point3 = client.point.get_by_path("\\\\PI1\SLA5.Turbine.fault prescriptive prediction", None)

def upload_recommendation(batch_data):

    streamValue1 = PIStreamValues()
    streamValue2 = PIStreamValues()
    streamValue3 = PIStreamValues()

    values1 = list()
    values2 = list()
    values3 = list()

    value1 = PITimedValue()
    value2 = PITimedValue()
    value3 = PITimedValue()

    for data in batch_data:
        timestamp = data[0]
        fault = data[1]
        priority = data[2]
        recommendation = data[3]
        
        value1.value = recommendation
        value1.timestamp = timestamp
        streamValue1.web_id = point1.web_id
        values1.append(value1)

        value2.value = priority
        value2.timestamp = timestamp
        streamValue2.web_id = point2.web_id
        values2.append(value2)

        value3.value = fault
        value3.timestamp = timestamp
        streamValue3.web_id = point3.web_id
        values3.append(value3)

    streamValue1.items = values1
    streamValue2.items = values2
    streamValue3.items = values3

    streamValues = list()
    streamValues.append(streamValue1)
    streamValues.append(streamValue2)
    streamValues.append(streamValue3)

    # response = client.streamSet.update_values_ad_hoc_with_http_info(streamValues, None, None)
    # return response

### Looping untuk waktu tertentu (2020 - 2021)
Dilakukan looping sampai `2021-12-31 23:59:00`

Jika ingin merubah akhir dari looping rubah value pada `current_time` di bawah

```python
if current_time == '2022-01-01 00:00:00':
    break
```

In [11]:
# looping for each minute
current_time = '2020-04-04 17:30:00'
temp_upload = []

while True:
    print(current_time)
    if current_time == '2020-04-04 17:31:00':
        break
    
    # get data
    data = etl(paths, current_time, current_time, interval)
    
    # load model (state matrix)
    model = Model(current_time)
    state_matrix = model.load_state_matrix()
    
    # estimate with VBM
    vbm = VBM(actual_high, actual_low)
    estimates, state_matrix = vbm.estimate_sensors(data['actuals'], state_matrix)
    
    # update model (state matrix)
    model.update_state_matrix(state_matrix)
    
    # calculate residual
    residual_indication_positives = []
    residual_indication_negatives = []
    residuals = []
    for i in range(len(data['actuals'])):
        resid = Residual(data['actuals'][i], estimates[i], residual_positive_threshold[i], residual_negative_threshold[i])
        residuals.append(resid.residual)
        residual_indication_positives.append(resid.residual_indication_positive)
        residual_indication_negatives.append(resid.residual_indication_negative)
    
    # insert to table runtime
    ## constructing the output
    values = []
    for i in range(len(sensor)):
        val = (current_time, sensor_id[i], fault, data["actuals"][i], None, estimates[i], residuals[i], None, residual_indication_positives[i], residual_indication_negatives[i])
        values.append(val)
    # insert into table runtime
    insert_to_table_runtime(values)
    
    # diagnostic process
    # get all diagnostic rule expression from the fault
    diag_rule = session.query(Fault).get(fault).diagnostics.all()
    # get spesific based on asset_id
    for d in diag_rule:
        if d.get_asset() == asset.id:
            diag = d
    # determine priority
    current_time = datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
    priority = diag.calculate(current_time, fault)
    if int(priority) > 0:
        # print("=== FAILURE DETECTED ===")
        fault_object = AssetFailure(date=current_time, 
                                    asset_id=asset.id,
                                    fault=diag.fault,
                                    priority=int(priority))
        # session.add(fault_object)
        # session.commit()

        recommendation = session.query(Priority.recomendation).filter_by(priority=priority, fault=fault).first()[0]
        # print(f'Recommendation: {recommendation}')
        # upload to PI: timestamp, fault, priority, rekomendasi
        
        # check apakah hari baru
        # if current_time.strftime('%Y-%m-%d %H:%M:%S').split()[1] == '00:00:00':
        #     status = upload_recommendation(temp_upload)
        #     print('batch data telah diupload ke pi system!')
        #     temp_upload.clear()
        # else:
        #     temp_upload.append((current_time, fault, priority, recommendation))
        
    # update diagnose_date
    # dgdate = session.query(DiagnoseDate).first()
    # dgdate.timestamp = current_time
    # session.commit()
    
    # update current time
    current_time = current_time + timedelta(minutes=1)
    current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')
    print(values)
    # print()

2020-04-04 17:30:00
[('2020-04-04 17:30:00', 1, 'Local Bearing', nan, None, nan, nan, None, 0, 0), ('2020-04-04 17:30:00', 2, 'Local Bearing', nan, None, nan, nan, None, 0, 0), ('2020-04-04 17:30:00', 3, 'Local Bearing', nan, None, nan, nan, None, 0, 0), ('2020-04-04 17:30:00', 5, 'Local Bearing', nan, None, nan, nan, None, 0, 0), ('2020-04-04 17:30:00', 8, 'Local Bearing', nan, None, nan, nan, None, 0, 0), ('2020-04-04 17:30:00', 9, 'Local Bearing', nan, None, nan, nan, None, 0, 0)]
2020-04-04 17:31:00


### Looping untuk real-time
Dilakukan looping secara real-time, hanya melakukan proses tiap menit.

> Catatan: harus sudah dilakukan running sampai waktu saat ini pada looping di atas.

In [ ]:
# looping for each minute
current_time = DiagnoseDate._get_last_diagnose() #get last diagnose date from db

while True:
    now_time = datetime.now().strftime('%Y-%m-%d %H:%M:00')
    # check waktu pada current_time apakah sama dengan now_time (waktu saat ini)
    # jika iya, tidak melakukan apa-apa (menunggu).
    # jika tidak, melakukan proses diagnostic hingga update current_time + 1 menit.
    if current_time == now_time:
        time.sleep(5)
        continue
    
    # get data
    data = etl(paths, current_time, current_time, interval)
    
    # load model (state matrix)
    model = Model(current_time)
    state_matrix = model.load_state_matrix()
    
    # estimate with VBM
    vbm = VBM(actual_high, actual_low)
    estimates, state_matrix = vbm.estimate_sensors(data['actuals'], state_matrix)
    
    # update model (state matrix)
    model.update_state_matrix(state_matrix)
    
    # calculate residual
    residual_indication_positives = []
    residual_indication_negatives = []
    residuals = []
    for i in range(len(data['actuals'])):
        resid = Residual(data['actuals'][i], estimates[i], residual_positive_threshold[i], residual_negative_threshold[i])
        residuals.append(resid.residual)
        residual_indication_positives.append(resid.residual_indication_positive)
        residual_indication_negatives.append(resid.residual_indication_negative)
    
    # insert to table runtime
    ## constructing the output
    values = []
    for i in range(len(sensor)):
        val = (current_time, sensor_id[i], fault, data["actuals"][i], None, estimates[i], residuals[i], None, residual_indication_positives[i], residual_indication_negatives[i])
        values.append(val)
    # insert into table runtime
    insert_to_table_runtime(values)
    
    # diagnostic process
    # get all diagnostic rule expression from the fault
    diag_rule = session.query(Fault).get(fault).diagnostics.all()
    # get spesific based on asset_id
    for d in diag_rule:
        if d.get_asset() == asset.id:
            diag = d
    # determine priority
    current_time = datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
    priority = diag.calculate(current_time, fault)
    if int(priority) > 0:
        # print("=== FAILURE DETECTED ===")
        fault_object = AssetFailure(date=current_time, 
                                    asset_id=asset.id,
                                    fault=diag.fault,
                                    priority=int(priority))
        session.add(fault_object)
        session.commit()
        # recommendation
        recommendation = session.query(Priority.recomendation).filter_by(priority=priority, fault=fault).first()[0]
        # print(f'Recommendation: {recommendation}')
        # upload to PI: timestamp, fault, priority, rekomendasi
        
        # check apakah hari baru
        if current_time.strftime('%Y-%m-%d %H:%M:%S').split()[1] == '00:00:00':
            status = upload_recommendation(temp_upload)
            # print('batch data telah diupload ke pi system!')
            temp_upload.clear()
        else:
            temp_upload.append((current_time, fault, priority, recommendation))
        
    # update diagnose_date
    dgdate = session.query(DiagnoseDate).first()
    dgdate.timestamp = current_time
    session.commit()
    
    # update current time
    current_time = current_time + timedelta(minutes=1)
    current_time = current_time.strftime('%Y-%m-%d %H:%M:%S')